In [ ]:
import json
import boto3
import pandas as pd
import os
import io
from io import BytesIO

In [2]:
RAW_BUCKET = "raw-data"
AWS_REGION = "us-east-1"
ENDPOINT_URL = "http://localhost:4566" # LocalStack endpoint

In [3]:
s3 = boto3.client("s3", endpoint_url=ENDPOINT_URL, region_name=AWS_REGION)

In [4]:
raw_data = "raw-data"
processed_bucket = "processed-data"

In [5]:
csv_objects = s3.list_objects_v2(Bucket=raw_data).get("Contents", [])
csv_files = [obj["Key"] for obj in csv_objects if obj["Key"] != "population_data.json"]

In [6]:
csv_dataframes = []
for key in csv_files:
    obj = s3.get_object(Bucket=raw_data, Key=key)
    df = pd.read_csv(obj["Body"], sep='\t')
    csv_dataframes.append(df)

In [8]:
if not csv_dataframes:
    print("No CSV files found in raw bucket.")
    csv_df = pd.DataFrame()
else:
    csv_df = pd.concat(csv_dataframes, ignore_index=True)
    
    
display(csv_df)

,series_id,year,period,value,footnote_codes,sector_code,class_code,measure_code,duration_code,seasonal,base_year,begin_year,begin_period,end_year,end_period
0,PRS30006011,1995.0,Q01,2.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PRS30006011,1995.0,Q02,2.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PRS30006011,1995.0,Q03,0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PRS30006011,1995.0,Q04,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PRS30006011,1995.0,Q05,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37516,PRS88003192,NaN,NaN,NaN,NaN,8800.0,3.0,19.0,2.0,S,-,1947.0,Q02,2025.0,Q02
37517,PRS88003193,NaN,NaN,NaN,NaN,8800.0,3.0,19.0,3.0,S,2017,1947.0,Q01,2025.0,Q02
37518,PRS88003201,NaN,NaN,NaN,NaN,8800.0,3.0,20.0,1.0,S,-,1948.0,Q01,2025.0,Q02
37519,PRS88003202,NaN,NaN,NaN,NaN,8800.0,3.0,20.0,2.0,S,-,1947.0,Q02,2025.0,Q02


In [9]:
json_objects = s3.list_objects_v2(Bucket=raw_data).get("Contents", [])
json_files = [obj["Key"] for obj in json_objects if obj["Key"] == "population_data.json"]
print(json_files)

['population_data.json']


In [10]:
json_dataframes = []
for key in json_files:
    obj = s3.get_object(Bucket=raw_data, Key=key)
    json_content = json.load(obj["Body"])
    
    if isinstance(json_content, dict):
        # Try common nested key like 'data' or 'records'
        if 'data' in json_content:
            df = pd.DataFrame(json_content['data'])
        elif 'records' in json_content:
            df = pd.DataFrame(json_content['records'])
        else:
            # Fallback: flatten dict of lists
            df = pd.json_normalize(json_content)
    elif isinstance(json_content, list):
        df = pd.DataFrame(json_content)
    else:
        raise ValueError("Unsupported JSON structure")

    json_dataframes.append(df)
    

In [13]:
if not json_dataframes:
    print("No JSON files found in processed bucket.")
    json_df = pd.DataFrame()
else:
    json_df = pd.concat(json_dataframes, ignore_index=True)
display(json_df)

,Nation ID,Nation,Year,Population
0,01000US,United States,2013,316128839.0
1,01000US,United States,2014,318857056.0
2,01000US,United States,2015,321418821.0
3,01000US,United States,2016,323127515.0
4,01000US,United States,2017,325719178.0
5,01000US,United States,2018,327167439.0
6,01000US,United States,2019,328239523.0
7,01000US,United States,2021,331893745.0
8,01000US,United States,2022,333287562.0
9,01000US,United States,2023,334914896.0


In [14]:
if not json_df.empty:
    pop_df = json_df.copy()
    pop_df = pop_df[(pop_df["Year"] >= 2013) & (pop_df["Year"] <= 2018)]
    mean_population = pop_df["Population"].mean()
    std_population = pop_df["Population"].std()
    print(f"Mean population (2013-2018): {mean_population}")
    print(f"Std deviation (2013-2018): {std_population}")

Mean population (2013-2018): 322069808.0
Std deviation (2013-2018): 4158441.040908095


In [15]:
print("\nColumn names and their indexes:")
for i, col in enumerate(csv_df.columns):
    print(f"{i}: {repr(col)}")


Column names and their indexes:
0: 'series_id        '
1: 'year'
2: 'period'
3: '       value'
4: 'footnote_codes'
5: 'sector_code'
6: 'class_code'
7: 'measure_code'
8: 'duration_code'
9: 'seasonal'
10: 'base_year'
11: 'begin_year'
12: 'begin_period'
13: 'end_year'
14: 'end_period'


In [16]:
# Clean up column names
csv_df.columns = (
    csv_df.columns
    .astype(str)
    .str.strip()          # remove surrounding spaces
    .str.replace("'", "") # remove quote characters
    .str.replace('"', '') # remove double quotes
)

print(csv_df.columns.tolist())

['series_id', 'year', 'period', 'value', 'footnote_codes', 'sector_code', 'class_code', 'measure_code', 'duration_code', 'seasonal', 'base_year', 'begin_year', 'begin_period', 'end_year', 'end_period']


In [17]:
# Strip whitespaces from column names
# csv_df.columns = csv_df.columns.str.strip()

# Confirm 'value' and 'series_id' exist after cleaning
if 'value' not in csv_df.columns or 'series_id' not in csv_df.columns:
    print("ERROR: Expected columns not found after stripping. Existing columns:")
    print(csv_df.columns.tolist())
else:
    print("Columns look fine. Proceeding with computation...\n")
    
    # Convert columns safely
    csv_df['value'] = pd.to_numeric(csv_df['value'], errors='coerce')
    csv_df['year'] = pd.to_numeric(csv_df['year'], errors='coerce')

    best_years = (
        csv_df.groupby('series_id')
              .apply(lambda x: x.groupby('year')['value'].sum().idxmax())
              .reset_index(name='best_year')
    )

    best_years['sum_value'] = best_years.apply(
        lambda row: csv_df[
            (csv_df['series_id'] == row['series_id']) &
            (csv_df['year'] == row['best_year'])
        ]['value'].sum(),
        axis=1
    )

    display(best_years)

Columns look fine. Proceeding with computation...



,series_id,best_year,sum_value
0,PRS30006011,2022.0,20.500
1,PRS30006012,2022.0,17.100
2,PRS30006013,1998.0,705.895
3,PRS30006021,2010.0,17.700
4,PRS30006022,2010.0,12.400
...,...,...,...
277,PRS88003192,2002.0,282.800
278,PRS88003193,2024.0,860.838
279,PRS88003201,2022.0,37.200
280,PRS88003202,2022.0,28.700


In [18]:
print("json_df columns:", json_df.columns.tolist())

json_df columns: ['Nation ID', 'Nation', 'Year', 'Population']


In [19]:
json_df.columns = (
    json_df.columns
    .astype(str)
    .str.strip()
    .str.replace("'", "")
    .str.replace('"', '')
    .str.lower()   # normalize to lowercase
)
display(json_df)

,nation id,nation,year,population
0,01000US,United States,2013,316128839.0
1,01000US,United States,2014,318857056.0
2,01000US,United States,2015,321418821.0
3,01000US,United States,2016,323127515.0
4,01000US,United States,2017,325719178.0
5,01000US,United States,2018,327167439.0
6,01000US,United States,2019,328239523.0
7,01000US,United States,2021,331893745.0
8,01000US,United States,2022,333287562.0
9,01000US,United States,2023,334914896.0


In [20]:
if not csv_df.empty and not json_df.empty:
    csv_df['series_id'] = csv_df['series_id'].str.strip()
    csv_df['period'] = csv_df['period'].str.strip()
    combined_df = csv_df[(csv_df['series_id'] == 'PRS30006032') & (csv_df['period'] == 'Q01')]
    combined_df = combined_df.merge(json_df[['year', 'population']], on='year', how='left')
    print("Combined report for PRS30006032, period Q01:")
    display(combined_df)

Combined report for PRS30006032, period Q01:


,series_id,year,period,value,footnote_codes,sector_code,class_code,measure_code,duration_code,seasonal,base_year,begin_year,begin_period,end_year,end_period,population
0,PRS30006032,1995.0,Q01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PRS30006032,1996.0,Q01,-4.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PRS30006032,1997.0,Q01,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PRS30006032,1998.0,Q01,0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PRS30006032,1999.0,Q01,-4.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PRS30006032,2000.0,Q01,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,PRS30006032,2001.0,Q01,-6.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,PRS30006032,2002.0,Q01,-6.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,PRS30006032,2003.0,Q01,-5.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,PRS30006032,2004.0,Q01,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
